# Web Scraping Exercise

## 1. Introduction and Planning

### Objective:
The goal of this exercise is to build a web scraper that collects data from a chosen website. You will learn how to send HTTP requests, parse HTML content, extract relevant data, and store it in a structured format.

### Tasks:
1. Identify the data you want to scrape.
2. Choose the target website(s).
3. Plan the structure of your project.

### Example:
For this exercise, we will scrape job listings from Indeed.com. We will extract job titles, company names, locations, and job descriptions.

## 2. Understanding the Target Website
### Objective:

Analyze the structure of the web pages to be scraped.
### Tasks:

* Inspect the target website using browser developer tools.
* Identify the HTML elements that contain the desired data.

### Instructions:

* Open your browser and navigate to the target website (e.g., Indeed.com).
* Right-click on the webpage and select "Inspect" or press Ctrl+Shift+I.
* Use the developer tools to explore the HTML structure of the webpage.
* Identify the tags and classes of the elements that contain the job titles, company names, locations, and descriptions.

## 3. Writing the Scraper
### Objective:

Develop the code to scrape data from the target website.
### Tasks:

* Send HTTP requests to the target website.
* Parse the HTML content and extract the required data.
* Handle pagination to scrape data from multiple pages.
* Implement error handling.

In [3]:
from bs4 import BeautifulSoup

# Load the HTML file
with open("data/deluxe-corned-beef-hash.html", "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Extracting the recipe title
title = soup.find("meta", {"property": "og:title"})["content"]

# Extracting the description
description = soup.find("meta", {"name": "description"})["content"]

# Extracting the ingredients
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
ingredients = [ingredient.get_text().strip() for ingredient in ingredients_section]

# Extracting the instructions
instructions_section = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
instructions = [instruction.get_text().strip() for instruction in instructions_section]

# Extracting the nutrition information
nutrition_section = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")
nutrition_facts = [fact.parent.get_text().strip().replace('\n', ' ') for fact in nutrition_section]

# Print the extracted information
print("Recipe Title:", title)
print("Description:", description)
print("Ingredients:")
for ingredient in ingredients:
    print("-", ingredient)
print("Instructions:")
for i, instruction in enumerate(instructions, 1):
    print(f"{i}. {instruction}")
print("Nutrition Facts:")
for fact in nutrition_facts:
    print("-", fact)


Recipe Title: Deluxe Corned Beef Hash
Description: This corned beef hash combines cooked corned beef with crispy potatoes and onions for a delicious way to use up leftover St. Paddy's Day corned beef.
Ingredients:
- 2 tablespoons butter
- 2 tablespoons extra-virgin olive oil
- 1 large onion, chopped
- 5 large Yukon Gold potatoes, peeled and cut into 1/4-inch cubes
- 1 large carrot, coarsely shredded
- 2 pounds cooked corned beef, cubed
- 2 tablespoons chopped fresh parsley
- ½ teaspoon ground black pepper, or to taste
- ¼ teaspoon dried thyme leaves
- 1 pinch salt to taste
Instructions:
1. Put your leftover corned beef to good use with this satisfying, hearty, and absolutely delicious corned beef hash recipe.
2. Corned beef hash is a hearty Irish meal made with potatoes and corned beef. The ingredients are traditionally pan-fried until they are tender and warm. Corned beef hash is a popular use for leftover corned beef (think the day after St. Patrick's Day).
3. You'll find the full, s

In [4]:
# Find all the links to other recipes
recipe_links = soup.find_all("a", href=True)

# Filter and print only the links that are likely to be recipes
recipe_urls = []
for link in recipe_links:
    href = link['href']
    if "recipe" in href:
        recipe_urls.append(href)

# Print the recipe URLs
print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/authentication/login?regSource=3675&relativeRedirectUrl=%2Frecipe%2F216981%2Fdeluxe-corned-beef-hash%2F
https://www.allrecipes.com/authentication/logout?relativeRedirectUrl=%2Frecipe%2F216981%2Fdeluxe-corned-beef-hash%2F
/account/add-recipe
https://www.magazines.com/allrecipes-magazine.html?utm_source=allrecipes.com&utm_medium=owned&utm_campaign=i111arr1w2661
https://www.magazines.com/allrecipes-magazine.html
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/94/soups-stews-and-chili/
https://www.allrecipes.com/recipes/16099/everyday-cooking/comfort-food/
https://www.allrecipes.com

In [5]:
import requests

def fetch_recipe_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract relevant information (title, ingredients, instructions, etc.)
        title = soup.find("meta", {"property": "og:title"})["content"] if soup.find("meta", {"property": "og:title"}) else "No title found"
        description = soup.find("meta", {"name": "description"})["content"] if soup.find("meta", {"name": "description"}) else "No description found"
        
        ingredients_section = soup.find_all("span", class_="recipe-ingredients__ingredient")
        ingredients = [ingredient.get_text(strip=True) for ingredient in ingredients_section] if ingredients_section else ["No ingredients found"]

        instructions_section = soup.find_all("li", class_="mntl-sc-block-group--LI mntl-sc-block mntl-sc-block-html")
        instructions = [instruction.get_text(strip=True) for instruction in instructions_section] if instructions_section else ["No instructions found"]
        
        # Print the extracted information for each recipe
        print("Recipe Title:", title)
        print("Description:", description)
        print("Ingredients:")
        for ingredient in ingredients:
            print("-", ingredient)
        print("Instructions:")
        for i, instruction in enumerate(instructions, 1):
            print(f"{i}. {instruction}")
        print("\n")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")

# Loop through the list of recipe URLs and fetch their contents
for url in recipe_urls[:10]:
    fetch_recipe_content(url)

Recipe Title: No title found
Description: No description found
Ingredients:
- No ingredients found
Instructions:
1. No instructions found


Recipe Title: Deluxe Corned Beef Hash
Description: This corned beef hash combines cooked corned beef with crispy potatoes and onions for a delicious way to use up leftover St. Paddy's Day corned beef.
Ingredients:
- No ingredients found
Instructions:
1. No instructions found


Failed to fetch /account/add-recipe: Invalid URL '/account/add-recipe': No scheme supplied. Perhaps you meant https:///account/add-recipe?
Recipe Title: No title found
Description: No description found
Ingredients:
- No ingredients found
Instructions:
1. No instructions found


Recipe Title: No title found
Description: No description found
Ingredients:
- No ingredients found
Instructions:
1. No instructions found


Recipe Title: Dinner Recipes
Description: What's for dinner? Take the work out of searching for an answer to this question with these popular 5-star dinner recipes